In [1]:
import matplotlib as mpl
import os
import glob
import json
import pymworks
import re
import datautils
import copy
import math
import time

import multiprocessing as mp
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as pl
import cPickle as pkl
from cPickle import PicklingError

import scipy.stats as spstats
import utils as util
import process_datafiles as processd

import pprint

pp = pprint.PrettyPrinter(indent=4)

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/pymworks/protocol/__init__.py:15: UserWarning: networkx is required for pymworks.protocols.states
  warnings.warn('networkx is required for pymworks.protocols.states')
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/datautils/__init__.py:22: UserWarning: datautils.mongo failed to import with: No module named pymongo
  warnings.warn('datautils.mongo failed to import with: %s' % E)
utils.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/julianarhee/anaconda2/e

# Set source and get meta info

In [2]:
rootdir = '/n/coxfs01/behavior-data'
paradigm = 'threeport'

metadata = util.get_metadata(paradigm, rootdir=rootdir, create_meta=False)


Loading existing metadata: /n/coxfs01/behavior-data/threeport/metadata.pkl


In [4]:
import assign_phase as ph

# Assign phases to each datafile

In [6]:
cohort = 'AL'

In [21]:

defaults = ph.get_default_params(cohort)
defaults

{'depth_rotation': 0.0,
 'expected_depth_rotations': array([-60., -55., -50., -45., -40., -35., -30., -25., -20., -15., -10.,
         -5.,   0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,
         50.,  55.,  60.]),
 'expected_sizes': array([15. , 17.5, 20. , 22.5, 25. , 27.5, 30. , 32.5, 35. , 37.5, 40. ]),
 'fine_depth_rotations': array([-60., -55., -50., -45., -40., -35., -30., -25., -20., -15., -10.,
         -5.,   0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,
         50.,  55.,  60.]),
 'planar_rotation': 0.0,
 'size': 40.0,
 'standard_depth_rotations': array([-60., -45., -30., -15.,   0.,  15.,  30.,  45.,  60.])}

In [22]:
default_size = defaults['size']
default_depth_rotation = defaults['depth_rotation']
default_planar_rotation = defaults['planar_rotation']

expected_sizes = defaults['expected_sizes']
expected_drots = defaults['expected_depth_rotations']
expected_size_interval = np.diff(expected_sizes).mean()
expected_drot_interval = np.diff(expected_drots).mean()


In [19]:
# phase_lookup = {0: 'manual',
#                 1: 'default',
#                 2: 'size',
#                 3: 'depth_rotation',
#                 4: 'cross',
#                 5: 'size_and_depth_rotation',
#                 6: 'depth_and_planar_rotation',
#                 7: 'morph',
#                 8: 'newstimuli',
#                 9: 'fine_grained_size',
#                 10: 'fine_grained_depth_rotation',
#                 11: 'fine_grained_size_and_depth_rotation',
#                 12: 'transparency',
#                 13: 'clutter',
#                 -1: 'other'}

# default_size = 30
# default_depth_rotation = 0.
# default_planar_rotation = 0.

# if cohort in ['AK', 'AL', 'AM']:
#     expected_sizes = np.linspace(15, 40, 11.)
#     expected_drots = np.linspace(-60, 60, 25.)
#     default_size = 40
    
# elif cohort in ['AG', 'AJ']:
#     expected_sizes = np.linspace(15, 40, 6.)
#     expected_drots = np.linspace(-60, 60, 9.)


# expected_size_interval = np.diff(expected_sizes).mean()
# expected_drot_interval = np.diff(expected_drots).mean()


new_stim_descs = ['stimB', 'stimC', 'NewObjects']


# array(['', 'b', 'big', 'stimB', 'stimBlowerbound', 'stimClowerbound',
#        'stimBupperbound', 'stimCupperbound', 'small', 'stimC', 'c',
#        'stimCalwaysReward', 'alwaysReward', 'size', 'adeptrotl',
#        'bdeptrotr', 'deptrotl', 'adeptrotr', 'asize', 'bdeptrotl',
#        'background', 'deptrotr', 'balwaysReward', 'initial', 'deprotl',
#        'backgroundalwaysReward', 'bsize', 'morphs', 'aalwaysReward',
#        'empty', 'aasize', 'a', 'probeNewObjects', 'occluded', 'r', 'd',
#        'transparency', 'nominrt'], dtype=object)


In [469]:
#nfiles = len([f for f, g in animal_meta.groupby(['animalid', 'datasource'])])


In [8]:
defaults

{'depth_rotation': 0.0,
 'expected_depth_rotations': array([-60., -55., -50., -45., -40., -35., -30., -25., -20., -15., -10.,
         -5.,   0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,
         50.,  55.,  60.]),
 'expected_sizes': array([15. , 17.5, 20. , 22.5, 25. , 27.5, 30. , 32.5, 35. , 37.5, 40. ]),
 'fine_depth_rotations': array([-60., -55., -50., -45., -40., -35., -30., -25., -20., -15., -10.,
         -5.,   0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,
         50.,  55.,  60.]),
 'planar_rotation': 0.0,
 'size': 40.0,
 'standard_depth_rotations': array([-60., -45., -30., -15.,   0.,  15.,  30.,  45.,  60.])}

In [69]:
exclude_ixs = []
phasedata = []
animal_meta = metadata[metadata['cohort']== cohort]

animalid = 'AL16'
session = 20170815


In [70]:
animalid, session

('AL16', 20170815)

In [71]:
animal_meta = animal_meta[(animal_meta['animalid']==animalid) & (animal_meta['session']==session)]


In [72]:
animal_meta

,animalid,cohort,datasource,session,suffix
454,AL16,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170815,b
455,AL16,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170815,


In [34]:
mgroup = animal_meta.loc[455]
dfn = mgroup['datasource']
mgroup, dfn

(animalid                                                   AL16
 cohort                                                       AL
 datasource    /n/coxfs01/behavior-data/threeport/cohort_data...
 session                                                20170815
 suffix                                                         
 Name: 455, dtype: object,
 '/n/coxfs01/behavior-data/threeport/cohort_data/AL/raw/AL16_170815.mwk')

In [63]:
phase = -1
#(animalid, cohort, dsource, session, sfx), = mgroup.values

curr_trials, curr_flags, metainfo = util.parse_mw_file(dfn, create_new=False)

# if curr_trials is None or len(curr_trials)==0:
#     exclude_ixs.extend(mgroup.index.tolist())
#     continue

sizes = sorted(np.unique([t['size'] for t in curr_trials]))
drots = sorted(np.unique([t['depth_rotation'] for t in curr_trials]))
prots = sorted(np.unique([t['rotation'] for t in curr_trials]))

tested_size_interval = np.median(np.diff(sizes))
tested_drot_interval = np.median(np.diff(drots))

protocol = metainfo['protocol']
experiment_folder = metainfo['experiment']

alpha_values = []
if any(['alpha_multiplier' in s.keys() for s in curr_trials]):
    alpha_values = np.unique([s['alpha_multiplier'] for s in curr_trials])


In [65]:
if len(sizes) > 1 and (len(drots)==1 and len(prots)==1):
    #if metainfo['protocol'] in ['Staircase through shape parameters', '']:        
    if ( (expected_sizes==sizes) is False):
        if (tested_size_interval < expected_size_interval):
            phase = 9
        elif (tested_size_interval==expected_size_interval):
            if (len(sizes) > len(expected_sizes)-1):
                # Probably just screwed up indexing
                phase = 2
            elif all([d in expected_sizes for d in sizes]) : 
                # only testing a subset of the expected values
                phase = 2
    elif all(expected_sizes==sizes):
        print("h")
        phase = 2
print(phase)

2


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
drots

[0]

In [75]:
# exclude_ixs = []
# phasedata = []
# animal_meta = metadata[metadata['cohort']== cohort]

for (animalid, dfn), mgroup in animal_meta.sort_values(by=['animalid', 'session']).groupby(['animalid', 'datasource']):
    
    phase = -1
    (animalid, cohort, dsource, session, sfx), = mgroup.values

    curr_trials, curr_flags, metainfo = util.parse_mw_file(dfn, create_new=False)
    
    if curr_trials is None or len(curr_trials)==0:
        exclude_ixs.extend(mgroup.index.tolist())
        continue
        
    sizes = sorted(np.unique([t['size'] for t in curr_trials]))
    drots = sorted(np.unique([t['depth_rotation'] for t in curr_trials]))
    prots = sorted(np.unique([t['rotation'] for t in curr_trials]))
    
    tested_size_interval = np.median(np.diff(sizes))
    tested_drot_interval = np.median(np.diff(drots))


    protocol = metainfo['protocol']
    experiment_folder = metainfo['experiment']
    
    alpha_values = []
    if any(['alpha_multiplier' in s.keys() for s in curr_trials]):
        alpha_values = np.unique([s['alpha_multiplier'] for s in curr_trials])
    
#     if session == 20180625:
#         break

    if 'morph' in protocol or any(['morph' in s['name'] for s in S.trials]):
        phase = 7
    
    elif 'newstim' in metainfo['experiment'] or any([descr in sfx for descr in new_stim_descs]):
        phase = 8

    elif len(alpha_values) > 1 or any([a != 1 for a in alpha_values]):
        phase = 12

    elif any(['background' in desc for desc in [sfx, metainfo['experiment']]]):
        phase = 13
    
    elif 'occluded' in sfx:
        phase = 13
    
    # ====== PHASE 1 ====================================
    elif len(sizes) == 1 and len(drots) == 1 and len(prots)==1:
        #if protocol in ['Initiate VIsual Pres Protocol', 'Initiate Visual Pres Protocol', '']:
        if (sizes[0]==default_size and drots[0]==default_depth_rotation and prots[0]==default_planar_rotation):
            phase = 1
    
    # ====== PHASE 2 ====================================
    elif len(sizes) > 1 and (len(drots)==1 and len(prots)==1):
        #if metainfo['protocol'] in ['Staircase through shape parameters', '']:        
        if ( (expected_sizes==sizes) is False):
            if (tested_size_interval < expected_size_interval):
                phase = 9
            elif (tested_size_interval==expected_size_interval):
                if (len(sizes) > len(expected_sizes)-1):
                    # Probably just screwed up indexing
                    phase = 2
                elif all([d in expected_sizes for d in sizes]) : 
                    # only testing a subset of the expected values
                    phase = 2
        elif all(expected_sizes==sizes):
            phase = 2
            
    # ====== PHASE 3 ====================================
    elif len(drots) > 1 and (len(sizes)==1 and len(prots)==1):
        #if metainfo['protocol'] in ['Staircase through shape parameters', '']:
        if ( (expected_drots==drots) is False):
            if (tested_drot_interval < expected_drot_interval):
                # Fine-grained spacing
                phase = 10
            elif (tested_drot_interval==expected_drot_interval):
                if (len(drots) > len(expected_drots)-1):
                    # Probably just screwed up indexing
                    phase = 3
                elif all([d in expected_drots for d in drots]): 
                    # only testing a subset of the expected values
                    phase = 3
        elif all(expected_drots==drots):
            phase = 3
            
    # ====== PHASE 4/5 ====================================
    elif len(drots) > 1 and len(sizes) > 1 and len(prots) == 1:
        #if metainfo['protocol'] in ['Test all transformations', '']:
        off_cross_transforms = list(set([(t['size'], t['depth_rotation']) for t in curr_trials \
                                         if t['size']!=default_size \
                                         and t['depth_rotation']!=default_depth_rotation]))
        
        if ( (expected_drots==drots) is False) or ( (expected_sizes==sizes) is False):
            if (tested_size_interval < expected_size_interval) or (tested_drot_interval < expected_drot_interval):
                # Fine-grained size/rotation transformations
                phase = 11
#             elif (len(drots) == len(expected_drots)-1) or (len(sizes) == len(expected_sizes)-1):
#                 # Probably just screwed up indexing
#                 if len(off_cross_transforms) == 0:
#                     phase = 4 if len(off_cross_transforms) == 0 else 5
#             elif ((expected_drots==drots) is False):
#                 if all(sizes==expected_sizes):
#                     # Another screw up, just not changing 1 of the depth-rot sides
#                     phase = 4 if len(off_cross_transforms)==0 else 5
#             elif ((expected_sizes==sizes) is False):
#                 if all(drots==expected_drots):
#                     # Another screw up, just not changing 1 of the sizes
#                     phase = 4 if len(off_cross_transforms)==0 else 5       
            elif all([d in expected_drots for d in drots]) and all([s in expected_sizes for s in sizes]):
                phase = 4 if (len(off_cross_transforms)==0) else 5
        else:
            # STANDARD
            if all(drots==expected_drots) and all(sizes==expected_sizes):
                phase = 4 if (len(off_cross_transforms)==0) else 5

    # ====== PHASE 6 (in-plane) ====================================
    elif len(prots) > 1 and len(sizes)==1:
        if metainfo['protocol'] == 'Test all transformations':
            phase = 6
    
    else:
        print("NO CLUE.")
        print("[%s, %s%s] unknown protocol: %s" % (animalid, session, suffix, protocol) )

        print("Size", sizes)
        print("D-rots", drots)
        print("P-rots", prots)
    
    print(animalid, '%s%s' % (session, sfx), phase)
    mgroup['phase'] = [phase for _ in np.arange(0, len(mgroup))]
    mgroup['protocol'] = [protocol for _ in np.arange(0, len(mgroup))]
    mgroup['experiment'] = [experiment_folder for _ in np.arange(0, len(mgroup))]
    
    phasedata.append(mgroup)
    
phasedata = pd.concat(phasedata, axis=0)


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipyk

('AL16', '20170815', 2)
('AL16', '20170815b', 5)


/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [497]:
phase_dfile = os.path.join(rootdir, paradigm, 'processed', 'meta', 'phases_%s.pkl' % cohort)
with open(phase_dfile, 'wb') as f:
    pkl.dump(phasedata, f, protocol=pkl.HIGHEST_PROTOCOL)
        

In [500]:
phase_dfile

'/n/coxfs01/behavior-data/threeport/processed/meta/phases_AK.pkl'

In [501]:
phasedata['phase'].unique()

array([ 1,  2,  3,  5, -1, 12])

In [502]:
phasedata.groupby(['animalid', 'phase']).count()

cohort  datasource  session  suffix  protocol  experiment
animalid phase                                                           
AK1      -1          1           1        1       1         1           1
          1          8           8        8       8         8           8
          2          2           2        2       2         2           2
          3          2           2        2       2         2           2
          5         90          90       90      90        90          90
          12        11          11       11      11        11          11
AK10      1         11          11       11      11        11          11
          2          3           3        3       3         3           3
          3         10          10       10      10        10          10
          5         88          88       88      88        88          88
AK11      1         14          14       14      14        14          14
          2          2           2        2       2         2           2
          3          4           4        4       4         4           4
          5         91          91       91      91        91          91
AK12     -1          1           1        1       1         1           1
          1          8           8        8       8         8           8
          2          2           2        2       2         2           2
          3          2           2        2       2         2           2
          5         91          91       91      91        91          91
AK2      -1          2           2        2       2         2           2
          1         18          18       18      18        18          18
          2          7           7        7       7         7           7
          3          5           5        5       5         5           5
          5         75          75       75      75        75          75
          12        11          11       11      11        11          11
AK3      -1          4           4        4       4         4           4
          1          7           7        7       7         7           7
          2          1           1        1       1         1           1
          3          1           1        1       1         1           1
          5         92          92       92      92        92          92
...                ...         ...      ...     ...       ...         ...
AK4       1          9           9        9       9         9           9
          2          3           3        3       3         3           3
          3          1           1        1       1         1           1
          5         92          92       92      92        92          92
AK5      -1          1           1        1       1         1           1
          1         10          10       10      10        10          10
          2          1           1        1       1         1           1
          3          2           2        2       2         2           2
          5         87          87       87      87        87          87
          12        11          11       11      11        11          11
AK6      -1          2           2        2       2         2           2
          1         18          18       18      18        18          18
          2          2           2        2       2         2           2
          3          7           7        7       7         7           7
          5         70          70       70      70        70          70
          12        11          11       11      11        11          11
AK7      -1          3           3        3       3         3           3
          1         14          14       14      14        14          14
          2          2           2        2       2         2           2
          3          4           4        4       4         4           4
          5         80          80       80      80        80          80
          12        

In [504]:
phasedata[phasedata['phase']==-1].groupby(['protocol', 'experiment']).count()

animalid  \
protocol                           experiment                                                     
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...         2   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         1   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...         5   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...         9   

                                                                                       cohort  \
protocol                           experiment                                                   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...       2   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       1   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       5   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...       9   

                                                                                       datasource  \
protocol                           experiment                                                       
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...           2   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...           1   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...           5   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...           9   

                                                                                       session  \
protocol                           experiment                                                    
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...        2   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...        1   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...        5   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...        9   

                                                                                       suffix  \
protocol                           experiment                                                   
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...       2   
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       1   
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...       5   
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...       9   

                                                                                       phase  
protocol                           experiment                                                 
Initiate Visual Pres Protocol      _Users_labuser_Documents_MWorks_behavior_protoc...      2  
                                   _Users_labuser_Downloads_RatBehaviorGeneral_Beh...      1  
Staircase through shape parameters _Users_labuser_Downloads_RatBehaviorGeneral_Beh...      5  
Test all transformations           _Users_labuser_Documents_MWorks_behavior_protoc...      9

In [ ]:
# check if this is in excluded:
# ABORTED:  /n/coxfs01/behavior-data/threeport/cohort_data/AL/raw/AL4_170919.mwk
# 


In [7]:
include_ixs = [i for i in animal_meta.index.tolist() if i not in exclude_ixs]
print(len(exclude_ixs), len(include_ixs), animal_meta.shape)


(13, 1498, (1511, 5))


In [547]:
def check_against_manual_sorting(animalid, phase, paradigm='threeport', 
                                 rootdir='/n/coxfs01/behavior-data'):
    '''
    Based on JM's sorting, expects:
        PHASE:
            2_size
            3_deptrotr_l
            4_fullcross, etc.
            
        os.path.join(by_phase_cohort_dir, 'AL13 - AL16', <PHASE>, AL*_YYMMDD.mwk)
    
    '''
    
    included_sessions =  None
    sorted_dfns = None
    
    cohort = re.search('(\D+)', animalid).group(0)
    sorted_cohort_dir = os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort, 'sorted', '_data')

    if cohort in ['AM', 'AN']:
        # 'PHASE_description'
        sorted_dfns = sorted(glob.glob(os.path.join(sorted_cohort_dir, '%i_*' % phase, \
                                                    '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AL':
        sorted_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_phase', '%s*' % cohort, 
                                                          '%i_*' % phase, '%s*' % cohort, 
                                                          '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AK':
        if phase in [4, 5]:
            phase_dir_str = '*full*'
        else:
            phase_dir_str = '%i_*' % phase

        sorted_dfns = sorted(glob.glob(os.path.join(sorted_cohort_dir, 'regular*', phase_dir_str, \
                                                            '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
        sorted_dfns.extend(glob.glob(os.path.join(sorted_cohort_dir, 'regular*', phase_dir_str, \
                                                            '%s_*.mwk' % animalid)))


        extra_dfns = glob.glob(os.path.join(sorted_cohort_dir, phase_dir_str,
                                                   '%s*' % cohort, '%s_*.mwk' % animalid))
        extra_dfns2 = glob.glob(os.path.join(sorted_cohort_dir, phase_dir_str, phase_dir_str,
                                                   '%s*' % cohort, '%s_*.mwk' % animalid))

        sorted_dfns.extend(extra_dfns)
        sorted_dfns.extend(extra_dfns2)
        sorted_dfns = sorted(list(set(sorted_dfns)), key=natural_keys)
        
    elif cohort == 'AO':
        sorted_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort,
                                                    'sorted', '_data', '*', '%i_*' % phase, 
                                         '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
        extra_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort,
                                                    'sorted', '_data_test', '%i_*' % phase, 
                                         '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
        sorted_dfns.extend(extra_dfns)
        sorted_dfns = sorted(list(set(sorted_dfns)), key=natural_keys)

    elif cohort == 'AJ':
        sorted_dfns = sorted(glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort, 'phase%i' % phase,
                                         '%s*' % cohort, '%s_*.mwk' % animalid)), key=natural_keys)
    elif cohort == 'AG':
        if phase in [3, 4]:
            sorted_dfns = glob.glob(os.path.join(rootdir, paradigm, 'by_cohort', '%s_Cohort' % cohort,
                                     '%s phase%i' % (cohort, phase), '%s*' % cohort, '%s_*.mwk' % animalid))
        else:
            return None

    #print("[%s] %i datafiles for phase %i" % (animalid, len(sorted_dfns), phase))
    if sorted_dfns is not None:
        included_sessions = [str(i) for i in \
                             sorted(list(set([util.parse_datafile_name(dfn)[1] for dfn in sorted_dfns])), 
                                    key=natural_keys)]

    return included_sessions



In [548]:
cohort, animalid

('AK', 'AK1')

In [549]:
phase = 5
cohort = 'AK'

# excluded_animals = {3: ['AG12']}


check_these = dict((animalid, {}) for animalid in phasedata['animalid'].unique())
not_found = []
for animalid, pdata in phasedata[phasedata['phase']==phase].groupby(['animalid']):
    
    manual_list = check_against_manual_sorting(animalid, phase)
    if manual_list is None:
        not_found.append((phase, animalid))
    
#     manual_list = sorted([str('20%s' % os.path.splitext(os.path.split(fn)[1])[0].split('_')[-1]) \
#                           for fn in manual_sorted_dfiles], key=natural_keys)
    
    found_list = sorted(['%s%s' % (session, suffix) for (session, suffix), g in pdata.groupby(['session', 'suffix'])], key=natural_keys)
    print("%s: Manual %i, Auto %i " % (animalid, len(manual_list), len(found_list)))
    
    if manual_list != found_list:
        check_these[animalid].update({'not_in_manual': [i for i in found_list if i not in manual_list],
                                      'not_in_found': [i for i in manual_list if i not in found_list]})
        

AK1: Manual 59, Auto 90 
AK10: Manual 46, Auto 88 
AK11: Manual 49, Auto 91 
AK12: Manual 57, Auto 91 
AK2: Manual 43, Auto 75 
AK3: Manual 63, Auto 92 
AK4: Manual 61, Auto 92 
AK5: Manual 56, Auto 87 
AK6: Manual 44, Auto 70 
AK7: Manual 51, Auto 80 
AK8: Manual 6, Auto 5 
AK9: Manual 46, Auto 85 


In [550]:
not_found

[]

In [560]:
check_these['AK5']

{'not_in_found': [],
 'not_in_manual': ['20170627b',
  '20171107',
  '20171110',
  '20171111',
  '20171112',
  '20171113',
  '20171116',
  '20171117',
  '20171120',
  '20171130',
  '20171201',
  '20171205',
  '20171206',
  '20171207',
  '20171208',
  '20171210',
  '20171211',
  '20171212',
  '20171213',
  '20171214',
  '20180206',
  '20180207',
  '20180208',
  '20180209',
  '20180213',
  '20180214',
  '20180215',
  '20180220',
  '20181002nominrt',
  '20181003nominrt',
  '20181004nominrt']}

In [164]:
animalid = 'AL7'
# check_sessions = ['20170912'] 
check_sessions = check_these[animalid]['not_in_found'] # Not in auto-sorted

check_pdata = phasedata[ (phasedata['animalid']==animalid) & (phasedata['session'].isin(check_sessions))].copy()
for (prot, exp), g in check_pdata.groupby(['protocol', 'experiment']):
    if 'morph' in prot:
        print prot

exp = '_Users_labuser_Downloads_RatBehaviorGeneral_Behavior_3Dtransforms_2way_blobs'
check_pdata[check_pdata['experiment']==exp]['session'].unique()


array([20170727])

In [165]:
print(animalid)
check_these[animalid]

AL7


{'not_in_found': ['20170727'], 'not_in_manual': ['20170727b', '20170728b']}

In [166]:
check_pdata

,animalid,cohort,datasource,session,suffix,phase,protocol,experiment
1231,AL7,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170727,,2,Staircase through shape parameters,_Users_labuser_Downloads_RatBehaviorGeneral_Be...
1230,AL7,AL,/n/coxfs01/behavior-data/threeport/cohort_data...,20170727,b,3,Staircase through shape parameters,_Users_labuser_Downloads_RatBehaviorGeneral_Be...


In [455]:
metadata['suffix'].unique()

array(['', 'b', 'big', 'stimB', 'stimBlowerbound', 'stimClowerbound',
       'stimBupperbound', 'stimCupperbound', 'small', 'stimC', 'c',
       'stimCalwaysReward', 'alwaysReward', 'size', 'adeptrotl',
       'bdeptrotr', 'deptrotl', 'adeptrotr', 'asize', 'bdeptrotl',
       'background', 'deptrotr', 'balwaysReward', 'initial', 'deprotl',
       'backgroundalwaysReward', 'bsize', 'morphs', 'aalwaysReward',
       'empty', 'aasize', 'a', 'probeNewObjects', 'occluded', 'r', 'd',
       'transparency', 'nominrt'], dtype=object)

In [134]:
reload(util)
reload(ph)

utils.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/julianarhee/anaconda2/envs/behavior/lib/python2.7/site-package

<module 'assign_phase' from 'assign_phase.py'>

In [189]:
import assign_phase as ph

In [413]:

# animalid = 'AG2'
# session = 20151201 # newelevation (x_rotation)
# # session = 20160314 # newelevation-upsidedown '.../Blobs_Training_Rot_xy_flipped/Blob_N2_CamRot_y45_CamRot_x10_LighPos_x0_y-10_z-10.png',
# # session = 20160213  # new positions
# session = 20160217  # new positions


# animalid = 'AL10'
# # session = 20171205 #20171130 #20170918 #20170727 -- what is "bigstim"/"smallstim"?
# # protocol = Test all transformations
# #session = 20170728 - canonical
# session = 20170822

animalid = 'AK1'
session = 20180621 #20171212

# animalid = 'AL1'
# session = 20170719 # Always Reward

# animalid = 'AO6'
# session = 20180522 #20151214

session_meta = metadata[(metadata['animalid']==animalid) & (metadata['session']==session)]
S = util.Session(session_meta)
_, _, metainfo = S.get_trials(create_new=False)
metainfo

--> [1] of 1: /n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK1_180621_CBwrong_discard.mwk


{'address': '192.168.0.36',
 'experiment': '_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'experiment_path': '/var/folders/ct/9rl6yygn7474gnc092xp2mzc0000gp/T/MWorks/Experiment Cache/_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'protocol': 'Test all transformations',
 'screen': {'distance': 12.5,
  'height': 11.75,
  'refresh_rate_hz': 60.0,
  'units': 'inches',
  'width': 14.75},
 'server': 'Setup 18'}

In [414]:
metainfo

{'address': '192.168.0.36',
 'experiment': '_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'experiment_path': '/var/folders/ct/9rl6yygn7474gnc092xp2mzc0000gp/T/MWorks/Experiment Cache/_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs',
 'protocol': 'Test all transformations',
 'screen': {'distance': 12.5,
  'height': 11.75,
  'refresh_rate_hz': 60.0,
  'units': 'inches',
  'width': 14.75},
 'server': 'Setup 18'}

In [415]:
[]

[]

In [416]:
_ = ph.print_phase_lookup()

{   -1: 'other',
    0: 'always_reward',
    1: 'default',
    2: 'size',
    3: 'depth_rotation',
    4: 'cross',
    5: 'size_and_depth_rotation',
    6: 'depth_and_planar_rotation',
    7: 'morph',
    8: 'newstimuli',
    9: 'fine_grained_size',
    10: 'fine_grained_depth_rotation',
    11: 'fine_grained_size_and_depth_rotation',
    12: 'transparency',
    13: 'clutter',
    14: 'light_position',
    15: 'x_rotation',
    16: 'position',
    17: 'punishcycle_long',
    18: 'punishcycle_short',
    19: 'no_min_RT'}


In [417]:
reload(ph)

<module 'assign_phase' from 'assign_phase.pyc'>

In [379]:
animalid

'AM10'

In [380]:
session_meta = metadata[metadata['animalid']=='AM11']


In [381]:
p = ph.assign_phase_by_animal(animalid, session_meta, create_new=False)

('AM11', '20171207alwaysReward', 0)


assign_phase.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ameta['phase'] = [phase for _ in np.arange(0, len(ameta))]
assign_phase.py:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ameta['protocol'] = [protocol for _ in np.arange(0, len(ameta))]
assign_phase.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-c

('AM11', '20171208alwaysReward', 0)
('AM11', '20171209alwaysReward', 0)
('AM11', '20171210', 17)
('AM11', '20171211', 17)
('AM11', '20171212', 17)
('AM11', '20171212size', 17)
('AM11', '20171213size', 17)
('AM11', '20171214size', 17)
('AM11', '20180206deptrotl', 17)
('AM11', '20180207adeptrotr', 17)
('AM11', '20180207b', 17)
('AM11', '20180208', 17)
('AM11', '20180209', 17)
('AM11', '20180213', 17)
('AM11', '20180214', 17)
('AM11', '20180215', 17)
('AM11', '20180220', 17)
('AM11', '20180403', 17)
('AM11', '20180404', 17)
('AM11', '20180405', 17)
('AM11', '20180410', 17)
('AM11', '20180411', 17)
('AM11', '20180412', 17)
('AM11', '20180413', 17)
('AM11', '20180414', 17)
('AM11', '20180415', 17)
('AM11', '20180416', 17)
('AM11', '20180417', 17)
('AM11', '20180418', 17)
('AM11', '20180419', 17)
('AM11', '20180420', 17)
('AM11', '20180421', 17)
('AM11', '20180423', 17)
('AM11', '20180424', 17)
('AM11', '20180425', 17)
('AM11', '20180426', 17)
('AM11', '20180427', 17)
('AM11', '20180522', 17

In [202]:
p

,animalid,cohort,datasource,session,suffix,phase,protocol,experiment
4933,AG2,AG,/n/coxfs01/behavior-data/threeport/cohort_data...,20160217,,15,Test all transformations wih depth rotation in...,_Users_labuser_Downloads_RatBehaviorGeneral_Be...


In [418]:
suffix_values = np.unique([s['suffix'] for s in S.trials])
suffix_values

array([''], dtype='|S1')

In [419]:
for si, sfx in enumerate(suffix_values): #[ suffix_values ]):
    
    print("%i of %i -- %s" % (int(si+1), len(suffix_values), sfx))
    
    curr_trials = [s for s in S.trials if s['suffix']==sfx]

    off_cross_transforms = list(set([(t['size'], t['depth_rotation']) for t in curr_trials \
                                     if (t['size'] not in [30, 40] #defaults['size'] \
                                     and t['depth_rotation']!=0)])) #defaults['depth_rotation']) ]))
    
    print("off-cross:", off_cross_transforms)
    print("---")

    sizes = sorted(np.unique([t['size'] for t in curr_trials]))
    drots = sorted(np.unique([t['depth_rotation'] for t in curr_trials]))
    prots = sorted(np.unique([t['rotation'] for t in curr_trials]))

    print("sizes:", sizes)
    print("depth rots:", drots)
    print("planar rots:", prots)
    
    print("---")

    #phasedata[ (phasedata['animalid']==animalid) & (phasedata['session']==int(session)) ]
    
    light_positions = list(set([s['light_position'] for s in curr_trials]))
    print("Light:", light_positions)

1 of 1 -- 
('off-cross:', [])
---
('sizes:', [15.0, 20.0, 25.0, 30.0, 35.0, 40.0])
('depth rots:', [-60, -45, -30, -15, 0, 15, 30, 45, 60])
('planar rots:', [0.0])
---
('Light:', [None])


In [420]:
S.flags

{'FlagAddFreeRewardToEarnedReward': 0,
 'FlagAlwaysEnforceMaxSeqLeng': 1,
 'FlagAlwaysReward': 0,
 'FlagAutomaticBiasSuppression': 0,
 'FlagCorrectionTrials': 0,
 'FlagCueStimSound': 0,
 'FlagDiscardHalfOfTheTrials': 1,
 'FlagEnrichSomeStaircases': 0,
 'FlagLongPunishment': 1,
 'FlagPlayOutcomeSound': 1,
 'FlagRepeatedTrial': 0,
 'FlagSampUniformDistr': 0,
 'FlagShowOnlyTrainedAxes': 1,
 'FlagShowStimAfterResponse': 1,
 'FlagShowVisualStimuli': 1,
 'FlagStaircaseDeptRotLeft': -1,
 'FlagStaircaseDeptRotRight': 1,
 'FlagStaircaseHorizSeparation': 1,
 'FlagStaircasePosHL': 0,
 'FlagStaircasePosHR': 0,
 'FlagStaircasePosVD': 0,
 'FlagStaircasePosVU': 0,
 'FlagStaircaseProbFreeRew': 1,
 'FlagStaircaseRotACW': 1,
 'FlagStaircaseRotCW': 1,
 'FlagStaircaseSize': -1,
 'N_PunishmentCycles': 5,
 'TooFast_time': 350000,
 'run_bounds': [(5611776881546, 5619944848999)]}

In [421]:
alpha_values=[]
if any(['alpha_multiplier' in s.keys() for s in S.trials]):
    alpha_values = [s['alpha_multiplier'] for s in S.trials]

np.unique(alpha_values)

array([1.])

In [422]:
S.trials[0]

{'action': 'draw',
 'alpha_multiplier': 1.0,
 'depth_rotation': 0,
 'duration': 5531531,
 'file_hash': '3c0f591fcbbe5741dbb1b2982cee3e696603f72d',
 'filename': '/var/folders/ct/9rl6yygn7474gnc092xp2mzc0000gp/T/MWorks/Experiment Cache/_Users_labuser_Documents_MWorks_behavior_protocols_Behavior_3Dtransforms_2way_blobs/tmp/Blobs_TrainingRatsD1D2/Blob_N1_CamRot_y0.png',
 'light_position': None,
 'name': 'Blob_N1_CamRot_y0',
 'no_feedback': False,
 'object': '1',
 'outcome': 'success',
 'outcome_time': 5611905470937,
 'pos_x': 0.0,
 'pos_y': 0.0,
 'response': 'Announce_AcquirePort1',
 'response_time': 5611905470321,
 'rotation': 0.0,
 'size': 25.0,
 'size_x': 43.67499923706055,
 'size_y': 43.67499923706055,
 'suffix': '',
 'time': 5611904530068,
 'type': 'image',
 'x_rotation': None}

In [423]:
print(S.source[0])
df = pymworks.open(S.source[0])


/n/coxfs01/behavior-data/threeport/cohort_data/AK/raw/AK1_180621_CBwrong_discard.mwk


In [406]:
df.get_codec()

{0: '#codec',
 1: '#systemEvent',
 2: '#components',
 3: '#termination',
 4: '#allowAltFailover',
 5: '#state_system_mode',
 6: '#announceMessage',
 7: '#stimDisplayUpdate',
 8: '#experimentLoadProgress',
 9: '#loadedExperiment',
 10: '#announceStimulus',
 11: '#announceSound',
 12: '#announceCalibrator',
 13: '#requestCalibrator',
 14: '#announceCurrentState',
 15: '#announceTrial',
 16: '#announceBlock',
 17: '#announceAssertion',
 18: '#serverName',
 19: '#mainScreenInfo',
 20: '#warnOnSkippedRefresh',
 21: '#realtimeComponents',
 22: 'debuggerActive',
 23: 'debuggerRunning',
 24: 'debuggerStep',
 25: '#serialBridgeAddress',
 26: '#syringePumpPort',
 27: 'Adjusted_Metric_Correct_Strategy',
 28: 'GainFactorCorrectMetric',
 29: 'FlagAutomaticBiasSuppression',
 30: 'WatchResponseSide',
 31: 'WatchSuccessBias',
 32: 'Metric_Correct_Strategy',
 33: 'Metric_Biased_Strategy',
 34: 'Metric_Alternation_Strategy',
 35: 'FlagStaircaseHorizSeparation',
 36: 'WatchSuccessStair_HorizSep',
 37: 'H

In [407]:
df.get_events('N_PunishmentCycles')

[Event[code=200, name=N_PunishmentCycles, time=173645672848, value=21],
 Event[code=200, name=N_PunishmentCycles, time=173645716680, value=21],
 Event[code=200, name=N_PunishmentCycles, time=174566482629, value=21],
 Event[code=200, name=N_PunishmentCycles, time=186689416934, value=21],
 Event[code=200, name=N_PunishmentCycles, time=186689466772, value=21]]

In [408]:
tevs = sorted(df.get_events('TooFast_time'), key=lambda x: x.time)
tevs

[Event[code=240, name=TooFast_time, time=173645672876, value=350000],
 Event[code=240, name=TooFast_time, time=173645716718, value=350000],
 Event[code=240, name=TooFast_time, time=174566482760, value=350000],
 Event[code=240, name=TooFast_time, time=186689416981, value=350000],
 Event[code=240, name=TooFast_time, time=186689466811, value=350000]]

In [409]:
int(tevs[-1].value)


350000

In [123]:
sysevs = df.get_events('#systemEvent')  
len(sysevs)

6

In [125]:
for s in sysevs:
    print s.value
    print("---")

{'payload': {'experiment path': '/var/folders/bq/d_40rj9j1slfyy_dz1h7_7p80000gn/T/MWorks/Experiment Cache/_Users_labuser_Downloads_RatBehaviorGeneral_Behavior_3Dtransforms_2way_blobs', 'saved variables': ['AB2_140919_vars', 'AB2_140922_vars', 'AB2_140924_vars', 'AB2_140930_vars', 'AB2_141003_vars', 'AB2_141203_vars', 'AB5_140922_vars', 'AB6_140919_vars', 'AB6_140922_vars', 'AB6_140924_vars', 'AB6_140930_vars', 'AB6_141003_vars', 'AB6_141203_vars', 'AF2_150130_vars', 'AG2_150202_vars', 'AG2_150314_vars', 'AG2_150428', 'AG2_150428_vars', 'AG2_150512_vars', 'AG2_150608_vars', 'AG2_151123_vars', 'AG2_160127_vars', 'AG2_160216_vars', 'AG2_1606025_vars', 'AG2_160603_vars', 'AG2_160604_vars', 'AG2_160606_vars', 'AG2_160607', 'AG2_160607_vars', 'AG2_160608_vars', 'AG2_160609_vars', 'AG2_160610_vars', 'AG2_160613_vars', 'AG2_160614_vars', 'AG2_160615_vars', 'AG2_160616_vars', 'AG2_160617_vars', 'AG2_160618_vars', 'AG2_160620_vars', 'AG2_160621_vars', 'AG2_160622_vars', 'AG2_160623_vars', 'AG2_1

In [109]:
[v.time for v in sysevs if v.value['payload_type']==4013 and v.value['payload']['running']==1]

[2157920927435, 2167659853595]

In [112]:
(2167659853595 - 2157920927435)/1E6


9738.92616

In [187]:
list(set([s['x_rotation'] for s in curr_trials]))


[None]

In [188]:
list(set([s['rotation'] for s in curr_trials]))


[0.0]

In [189]:
list(set([(s['pos_x'], s['pos_y']) for s in curr_trials]))

[(0.0, 0.0)]

In [190]:
light_positions = list(set([s['light_position'] for s in curr_trials]))
#print(any([p is None for p in light_positions]))
light_positions




[None]

In [191]:
list(set([s['depth_rotation'] for s in curr_trials]))


[0, -30, -60, 45, 15, -15, -45, 60, 30]

In [192]:
list(set([s['size'] for s in curr_trials]))


[35.0, 40.0, 15.0, 20.0, 25.0, 30.0]

In [94]:
namestr = os.path.split(S.trials[0]['filename'])[-1]
namestr.split('CamRot_x')[1].split('_')

['10', 'LighPos', 'x0', 'y-10', 'z-10.png']

In [95]:
tuple([int(i) for i in re.findall("[-\d]+", namestr.split('LighPos')[1])])


(0, -10, -10)

In [441]:
phasedata.head()

AttributeError: 'list' object has no attribute 'head'

In [478]:
alpha_values = np.unique([s['alpha_multiplier'] for s in S.trials])
if len(alpha_values) > 1:
    phase = 12
elif any([a!=1 for a in alpha_values]):
    phase = 12

In [479]:
phase

12